In [16]:
from networkx import DiGraph, find_cycle, NetworkXNoCycle, ancestors
import types
from functools import reduce
from unification import unify, reify, variable, Var

In [17]:
# Add comparison method to the Var class so that variables can be sorted.
class Var(Var):
    def __lt__(self, y):
        if type(y) == Var:
            return self.token < y.token
        else:
            return NotImplemented

In [114]:
def expVars(e):
    """ Get all the variables in a tuple expression.
        Usage example:
            expVars(('imp', A(3), ('imp', B(1), C(0))))
        
        Output:
            {~A3, ~B1, ~C0}
    """
    if type(e) == tuple:
        return set.union(*map(expVars, e))
    elif type(e) == Var:
        return set([e])
    else:
        return set()

def substitutionGraph(s):
    """ Generate a digraph from a substitution, s.
    """
    g = DiGraph()
    g.add_edges_from(
        [ (s, t)
          for s, ts in zip(s.keys(), map(expVars, s.values()))
          for t in ts ] )
    return g

def directPairs(g):
    """ Gives the collection of pairs, (n, x), such that there's
        a chain of edges from n to x.
        
        Note: There's definitely a more efficient implementation, but
        I'm not sure if it's worth it.
    """
    return [ (n, x) for n in g.nodes for x in ancestors(g, n) ]

In [125]:
def unorderedPairs(vs):
    """ Take a list of things, and separate it into
        a list of distinct pairs.
        
        Example:
            unorderedPairs(['b', 'a', 'c'])
            
        Output:
            [('a', 'b'), ('a', 'c'), ('b', 'c')]
    """
    vs.sort()
    return [ (vs[i], x) for i in range(len(vs)-1) for x in vs[i+1:] ]

def DVSCGen(x):
    """ Given a list of lists of variables, this will generate valid 
        distinct variable side conditions to be used in a Rule.
        
        Example:
            DVSCGen([[Var('A'), Var('x')], [Var('B'), Var('x'), Var('y')]])
        
        Output:
            {(~A, ~x), (~B, ~x), (~B, ~y), (~x, ~y)}
    """
    return { p for l in map(unorderedPairs, x) for p in l }

def DVSCUpdate(d, s):
    """ Given a substitution rule, update a list of distinct variable
        side conditions. If the distinction condition is violated,
        this function returns False.
        
        Example usage 1:
            DVSCUpdate(
                {(~B0, ~x), (~B0, ~y), (~P, ~x), (~x, ~y)},
                {~P: ('mp', ~A0, ~B0, ~P0, ~Q0), ~B0: ('imp', 'phi', 'phi')}
                )
        Output:
            {(~A0, ~x), (~P0, ~x), (~Q0, ~x), (~x, ~y)}
            
        Example usage 2:
            DVSCUpdate(
                {(~A0, ~P), (~B0, ~x), (~B0, ~y), (~x, ~y)},
                {~P: ('mp', ~A0, ~B0, ~P0, ~Q0), ~B0: ('imp', 'phi', 'phi')}
                )
        
        Output:
            False
    """

    # Get pairs of sets of variables following substitution.
    varSets = [ (expVars(a), expVars(b)) for a, b in reify(list(d), s) ]

    # Test if distinctness condition is violated.
    # If two variables are distinct, then the sets of variables within
    # the result of substitution should have an empty intersection.
    if any([set.intersection(*p) for p in varSets]):
        return False

    # Split the pairs of sets into a set of pairs.
    return { (a, b) if a < b else (b, a) for v in varSets for a in v[0] for b in v[1] }


In [22]:
class Rule():
    """ Defines a rule, consisting of a single consequent
        and a list of hypotheses which imply this consequent.
       
        A theorem is a rule with an empty list of hypotheses.
        
        DVSC Represents the distinct variable side conditions,
        if they exist, as a list of sorted, ordered pairs.
    """
    
    def __init__(self, hypotheses, consequent, DVSC = set()):
        self.hypotheses = hypotheses
        self.consequent = consequent
        self.DVSC = DVSC

IntImpTheory

In [23]:
def A(x):
    """ Generate a variable 'Ax'
    """

    return Var( 'A' + str(x) )

def B(x):
    """ Generate a variable 'Bx'
    """

    return Var( 'B' + str(x) )

def C(x):
    """ Generate a variable 'Cx'
    """

    return Var( 'C' + str(x) )

def imp(x, y):
    return ('imp', x, y)

class IntImpTheory:
    """ Defines a basic theory of the intuitionistic
        implicational calculus.
       
        This theory has three rules:
       
        the axiom of simplification:
            a => (b => a)
            
        the axiom of frege:
            (a => (b => c)) => ((a => b) => (a => c))
        
        and modus ponens:
            a
            a => b
            ------
            b
           
        along with facilities for making their appropriate
        patterns, compatible with unification.
    """
    
    def __init__(self):
        self.freshCounter = 0
        self.rules = {'simplification': self.simplification,
                      'frege': self.frege,
                      'modusPonens': self.modusPonens
                     }
        
    def simplification(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([],
                    imp(A(i), imp(B(i), A(i))) )
        
    def frege(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([],
                    imp(imp(A(i), imp(B(i), C(i))),
                        imp(imp(A(i), B(i)),
                            imp(A(i), C(i)))) )
        
    def modusPonens(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ A(i), imp(A(i), B(i)) ],
                      B(i) )


IntImpProofTheory

In [24]:
def A(x):
    """ Generate a variable 'Ax'
    """

    return Var( 'A' + str(x) )

def B(x):
    """ Generate a variable 'Bx'
    """

    return Var( 'B' + str(x) )

def C(x):
    """ Generate a variable 'Cx'
    """

    return Var( 'C' + str(x) )

def P(x):
    """ Generate a variable 'Px'
    """

    return Var( 'P' + str(x) )

def Q(x):
    """ Generate a variable 'Qx'
    """

    return Var( 'Q' + str(x) )

def imp(x, y):
    return ('imp', x, y)

def proof(x, y):
    return ('proof', x, y)

def wff(x):
    return ('wff', x)

def simp(a, b):
    return ('simp', a, b)

def frege(a, b, c):
    return ('frege', a, b, c)

def mp(a, b, p1, p2):
    return ('mp', a, b, p1, p2)

class IntImpProofTheory:
    """ Defines a basic theory of the intuitionistic
        implicational calculus which also keeps track
        of proof terms.
       
        This theory has three rules:
       
        the axiom of simplification:
            a => (b => a)
            
        the axiom of frege:
            (a => (b => c)) => ((a => b) => (a => c))
        
        and modus ponens:
            a
            a => b
            ------
            b
           
        along with facilities for making their appropriate
        patterns, compatible with unification.
    """

    def __init__(self):
        self.freshCounter = 0
        self.rules = {
                      'phiWff': self.phiWff,
                      'psiWff': self.psiWff,
                      'chiWff': self.chiWff,
                      'thetaWff': self.thetaWff,
                      'zetaWff': self.thetaWff,
                      'etaWff': self.thetaWff,
                      'impWff': self.impWff,
                      'simplification': self.simplification,
                      'frege': self.frege,
                      'modusPonens': self.modusPonens
                     }

    def phiWff(self):
        return Rule([], wff('phi'))

    def psiWff(self):
        return Rule([], wff('psi'))

    def chiWff(self):
        return Rule([], wff('chi'))

    def thetaWff(self):
        return Rule([], wff('theta'))

    def zetaWff(self):
        return Rule([], wff('zeta'))

    def etaWff(self):
        return Rule([], wff('eta'))

    def impWff(self):
        """ Syntactic rule for determininte the well-formedness of
            an implicational formula.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ wff(A(i)), wff(B(i)) ],
                      wff(imp(A(i), B(i))) )

    def simplification(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ wff(A(i)), wff(B(i)) ],
                      proof(simp(A(i), B(i)), imp(A(i), imp(B(i), A(i))) ) )

    def frege(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ wff(A(i)), wff(B(i)), wff(C(i)) ],
                      proof(frege(A(i), B(i), C(i)),
                            imp(imp(A(i), imp(B(i), C(i))),
                                imp(imp(A(i), B(i)), imp(A(i), C(i)))) ) )

    def modusPonens(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ wff(A(i)),
                      wff(B(i)),
                      proof(P(i), A(i)),
                      proof(Q(i), imp(A(i), B(i))) ],
                      proof(mp(A(i), B(i), P(i), Q(i)), B(i)) )


LambdaTheory

In [ ]:
# Old version with explicit variables.

def x(i):
    """ Generate a variable 'xi'
    """

    return Var( 'x' + str(i) )

def y(x):
    """ Generate a variable 'yx'
    """

    return Var( 'y' + str(x) )

def A(x):
    """ Generate a variable 'Ax'
    """

    return Var( 'A' + str(x) )

def B(x):
    """ Generate a variable 'Bx'
    """

    return Var( 'B' + str(x) )

def C(x):
    """ Generate a variable 'Cx'
    """

    return Var( 'C' + str(x) )


def D(x):
    """ Generate a variable 'Dx'
    """

    return Var( 'D' + str(x) )

def P(x):
    """ Generate a variable 'Px'
    """

    return Var( 'P' + str(x) )

def Q(x):
    """ Generate a variable 'Qx'
    """

    return Var( 'Q' + str(x) )

def lam(x, y):
    return ('lam', x, y)

def app(x, y):
    return ('app', x, y)

def eq(x, y):
    return ('eq', x, y)

def ev(x):
    return ('ev', x)

def exp(a):
    return ('exp', a)

class LambdaTheory:
    """ A basic theory defining the evaluation of lambda
        expressions. Mainly used for testing distrinct
        variable side-conditions.
    """

    def __init__(self):
        self.freshCounter = 0
        self.rules = {
                      'varExp': self.varExp,
                      'xEv': self.xEv,
                      'yEv': self.yEv,
                      'zEv': self.zEv,
                      'uEv': self.uEv,
                      'vEv': self.vEv,
                      'wEv': self.wEv,
                      'XEv': self.XEv,
                      'YEv': self.YEv,
                      'ZEv': self.ZEv,
                      'UEv': self.UEv,
                      'VEv': self.VEv,
                      'WEv': self.WEv,
                      'lamExp': self.lamExp,
                      'appExp': self.appExp,
                      'transitivity': self.transitivity,
                      'symmetry': self.symmetry,
                      'equalityOfApplication': self.equalityOfApplication,
                      'equalityOfLambdas': self.equalityOfLambdas,
                      'etaEquality': self.etaEquality,
                      'beta1': self.beta1,
                      'beta2': self.beta2,
                      'beta3': self.beta3,
                      'beta4': self.beta4,
                      'beta5': self.beta5
                     }

    def varExp(self):
        """ An extentional variable is an expression
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(A(i)) ], exp(A(i)))

    # Extentional variables
    def xEv(self):
        return Rule([], ev('x'))

    def yEv(self):
        return Rule([], ev('y'))

    def zEv(self):
        return Rule([], ev('z'))

    def uEv(self):
        return Rule([], ev('u'))
    
    def vEv(self):
        return Rule([], ev('v'))

    def wEv(self):
        return Rule([], ev('w'))

    # Expression Variables
    def XEv(self):
        return Rule([], exp('X'))

    def YEv(self):
        return Rule([], exp('Y'))

    def ZEv(self):
        return Rule([], exp('Z'))

    def UEv(self):
        return Rule([], exp('U'))
    
    def VEv(self):
        return Rule([], exp('V'))

    def WEv(self):
        return Rule([], exp('W'))

    # Expression syntax
    def lamExp(self):
        """ Syntactic rule for determininte the well-formedness of
            a lambda expression.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), exp(B(i)) ],
                      exp(lam(x(i), B(i))) )

    def appExp(self):
        """ Syntactic rule for determininte the well-formedness of
            an application.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ exp(A(i)), exp(B(i)) ],
                      exp(app(A(i), B(i))) )
    
    
    # Equality Rules
    def transitivity(self):
        """ Transitivity of equality
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(A(i), B(i)), eq(B(i), C(i)) ],
                      eq(A(i), C(i)) )
    
    def symmetry(self):
        """ symmetry of equality
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(C(i), A(i)) ],
                      eq(A(i), C(i)) )
    
    def equalityOfApplication(self):
        """ Equality between applications
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(A(i), B(i)), eq(C(i), D(i)) ],
                      eq(app(A(i), C(i)), app(B(i), D(i))) )
    
    def equalityOfLambdas(self):
        """ Equality between lambda expressions.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), exp(B(i)), exp(C(i)), eq(B(i), C(i)) ],
                      eq(lam(x(i), B(i)), lam(x(i), C(i))) )
    
    def etaEquality(self):
        """ Extentional equality between expressions.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), eq(app(A(i), x(i)), app(B(i), x(i))) ],
                      eq(A(i), B(i)),
                      DVSCGen([[A(i), x(i)], [B(i), x(i)]])
                    )
    
    # Lambda expression reduction
    def beta1(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), exp(B(i)) ],
                      eq(app(lam(x(i), x(i)), B(i)), B(i)) )
    
    def beta2(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), exp(B(i)), exp(C(i)) ],
                      eq( app(lam(x(i), lam(x(i), B(i))), C(i)) , lam(x(i), B(i)))   )
    
    def beta3(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), ev(y(i)), exp(B(i)), exp(C(i)) ],
                      eq( app(lam(x(i), lam(y(i), B(i))), C(i)), lam(y(i), app(lam(x(i), B(i)), C(i))) ),
                      DVSCGen([[x(i), y(i)]])
                    )
    
    def beta4(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), exp(A(i)), exp(B(i)), exp(C(i)) ],
                      eq( app(lam(x(i), app(A(i), B(i))), C(i)),
                          app(app(lam(x(i), A(i)), C(i)), app(lam(x(i), B(i)), C(i)))
                      )
                     )
    
    def beta5(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ev(x(i)), exp(A(i)), exp(B(i)) ],
                      eq(app(lam(x(i), A(i)), B(i)), A(i)),
                      DVSCGen([[x(i), A(i)]])
                     )

In [176]:
def x(i):
    """ Generate a variable 'xi'
    """

    return Var( 'x' + str(i) )

def y(x):
    """ Generate a variable 'yx'
    """

    return Var( 'y' + str(x) )

def A(x):
    """ Generate a variable 'Ax'
    """

    return Var( 'A' + str(x) )

def B(x):
    """ Generate a variable 'Bx'
    """

    return Var( 'B' + str(x) )

def C(x):
    """ Generate a variable 'Cx'
    """

    return Var( 'C' + str(x) )


def D(x):
    """ Generate a variable 'Dx'
    """

    return Var( 'D' + str(x) )

def P(x):
    """ Generate a variable 'Px'
    """

    return Var( 'P' + str(x) )

def Q(x):
    """ Generate a variable 'Qx'
    """

    return Var( 'Q' + str(x) )

def lam(x, y):
    return ('lam', x, y)

def app(x, y):
    return ('app', x, y)

def eq(x, y):
    return ('eq', x, y)

def ev(x):
    return ('ev', x)

def exp(a):
    return ('exp', a)

class LambdaTheory:
    """ A basic theory defining the evaluation of lambda
        expressions. Mainly used for testing distrinct
        variable side-conditions.
    """

    def __init__(self):
        self.freshCounter = 0
        self.rules = {
                      'transitivity': self.transitivity,
                      'symmetry': self.symmetry,
                      'equalityOfApplication': self.equalityOfApplication,
                      'equalityOfLambdas': self.equalityOfLambdas,
                      'etaEquality': self.etaEquality,
                      'beta1': self.beta1,
                      'beta2': self.beta2,
                      'beta3': self.beta3,
                      'beta4': self.beta4,
                      'beta5': self.beta5
                     }

    # Equality Rules
    def transitivity(self):
        """ Transitivity of equality
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(A(i), B(i)), eq(B(i), C(i)) ],
                      eq(A(i), C(i)) )
    
    def symmetry(self):
        """ symmetry of equality
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(C(i), A(i)) ],
                      eq(A(i), C(i)) )
    
    def equalityOfApplication(self):
        """ Equality between applications
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(A(i), B(i)), eq(C(i), D(i)) ],
                      eq(app(A(i), C(i)), app(B(i), D(i))) )
    
    def equalityOfLambdas(self):
        """ Equality between lambda expressions.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(B(i), C(i)) ],
                      eq(lam(x(i), B(i)), lam(x(i), C(i))) )
    
    def etaEquality(self):
        """ Extentional equality between expressions.
        """
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ eq(app(A(i), x(i)), app(B(i), x(i))) ],
                      eq(A(i), B(i)),
                      DVSCGen([[A(i), x(i)], [B(i), x(i)]])
                    )
    
    # Lambda expression reduction
    def beta1(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ],
                      eq(app(lam(x(i), x(i)), B(i)), B(i)) )
    
    def beta2(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ],
                      eq( app(lam(x(i), lam(x(i), B(i))), C(i)) , lam(x(i), B(i)))   )
    
    def beta3(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ],
                      eq( app(lam(x(i), lam(y(i), B(i))), C(i)), lam(y(i), app(lam(x(i), B(i)), C(i))) ),
                      DVSCGen([[x(i), y(i)]])
                    )
    
    def beta4(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ],
                      eq( app(lam(x(i), app(A(i), B(i))), C(i)),
                          app(app(lam(x(i), A(i)), C(i)), app(lam(x(i), B(i)), C(i)))
                      )
                     )
    
    def beta5(self):
        i = self.freshCounter
        self.freshCounter += 1
        
        return Rule([ ],
                      eq(app(lam(x(i), A(i)), B(i)), A(i)),
                      DVSCGen([[x(i), A(i)]])
                     )

In [26]:
class ProofGame:

    """ Reperesents a game where the goal is to prove the
        hypothesis in the starting state.
    """

    def __init__(self, startingState, theory):
        self.startingState = startingState
        self.state = [startingState]
        self.DVSC = set([])
        self.theory = theory()
        # History consists of a list of ordered triples,
        # (state, substitutions, distinct variables)
        self.history = []

    def substitutions(self):
        """ Get the list of substitutions from
            the history.
        """ 
        
        return [ x[1] for x in self.history ]

    def finalState(self):
        """ Calculate the final state by applying
            all the built up substitutions to the
            starting state.
        """
        
        p = self.startingState
        for s in self.substitutions():
            p = reify(p, s)

        return p

    def currentGoal(self):
        """ The current goal which moves will target.
        """

        if len(self.state) > 0:
            return self.state[-1]
        else:
            return False

    def undo(self):
        """ Undoes the last action.
        """

        if len(self.history) > 0:
            self.state, _, self.DVSC = self.history.pop()
            return True
        
        return False

    def rotateLeft(self):
        """ Rotate the goal context to the left.
        """
        
        self.history.append((self.state, {}, self.DVSC))
        self.state = self.state[1:] + self.state[:1] 
        
        return True

    def rotateRight(self):
        """ Rotate the goal context to the right.
        """
        
        self.history.append((self.state, {}, self.DVSC))
        self.state = self.state[-1:] + self.state[:-1] 
        
        return True

    def applyRule(self, theoryRule):
        """ Attempts to unify the currentGoal
            with the conclusion of the provided rule
            and replace it with appropriately unified
            hypotheses of said rule.
              
            Will return "true" if successful,
            "false" otherwise.
        """

        goal = self.currentGoal()
        if goal:
            rule = self.theory.rules[theoryRule]()
            s = unify(goal, rule.consequent)
            if s != False:
                # Check for infinite loops.
                try:
                    find_cycle(substitutionGraph(s))
                    
                except NetworkXNoCycle:
                    
                    # Check that distinct variable side conditions
                    # are followed
                    newDVSC = DVSCUpdate(self.DVSC.union(rule.DVSC), s)
                    if newDVSC == False:
                        return False
                    
                    # Append old state to history.
                    self.history.append((self.state, s, self.DVSC))
                    
                    # Update distinct variables.
                    self.DVSC = newDVSC
                    
                    # Update state
                    self.state.pop()
                    self.state += rule.hypotheses
                    self.state = list(map(lambda g: reify(g, s), self.state))
                    
                    return True
        return False

In [665]:
unify(('imp', A(0), A(0)), ('imp', A(1), ('imp', B(0), A(1))))

{~A0: ~A1, ~A1: ('imp', ~B0, ~A1)}

In [205]:
id = ('imp', 'phi', 'phi')
idGame = ProofGame(id, IntImpTheory)

In [206]:
idGame.state

[('imp', 'phi', 'phi')]

In [207]:
idGame.applyRule('modusPonens')
idGame.state

[~A0, ('imp', ~A0, ('imp', 'phi', 'phi'))]

In [208]:
idGame.applyRule('modusPonens')
idGame.state

[~A0, ~A1, ('imp', ~A1, ('imp', ~A0, ('imp', 'phi', 'phi')))]

In [209]:
idGame.applyRule('frege')
idGame.state

[('imp', 'phi', ~B2), ('imp', 'phi', ('imp', ~B2, 'phi'))]

In [210]:
idGame.applyRule('simplification')
idGame.state

[('imp', 'phi', ~B3)]

In [211]:
idGame.applyRule('simplification')
idGame.state

[]

In [212]:
idGame.history

[([~A0, ('imp', ~A0, ~B0)], {~B0: ('imp', 'phi', 'phi')}, set()),
 ([~A0, ~A1, ('imp', ~A1, ~B1)],
  {~B1: ('imp', ~A0, ('imp', 'phi', 'phi'))},
  set()),
 ([~A0, ~A1],
  {~A1: ('imp', ~A2, ('imp', ~B2, ~C2)),
   ~A0: ('imp', ~A2, ~B2),
   ~A2: 'phi',
   ~C2: 'phi'},
  set()),
 ([('imp', 'phi', ~B2)], {~A3: 'phi', ~B2: ~B3}, set()),
 ([], {~A4: 'phi', ~B3: ('imp', ~B4, ~A4)}, set())]

In [138]:
idProofGame = ProofGame(proof(Var('P'), imp('phi', 'phi')), IntImpProofTheory)
idProofGame.state

[('proof', ~P, ('imp', 'phi', 'phi'))]

In [139]:
idProofGame.applyRule('modusPonens')
idProofGame.state

[('wff', ~A0),
 ('wff', ('imp', 'phi', 'phi')),
 ('proof', ~P0, ~A0),
 ('proof', ~Q0, ('imp', ~A0, ('imp', 'phi', 'phi')))]

In [140]:
idProofGame.applyRule('modusPonens')
idProofGame.state

[('wff', ~A0),
 ('wff', ('imp', 'phi', 'phi')),
 ('proof', ~P0, ~A0),
 ('wff', ~A1),
 ('wff', ('imp', ~A0, ('imp', 'phi', 'phi'))),
 ('proof', ~P1, ~A1),
 ('proof', ~Q1, ('imp', ~A1, ('imp', ~A0, ('imp', 'phi', 'phi'))))]

In [141]:
idProofGame.applyRule('frege')
idProofGame.state

[('wff', ('imp', 'phi', ~B2)),
 ('wff', ('imp', 'phi', 'phi')),
 ('proof', ~P0, ('imp', 'phi', ~B2)),
 ('wff', ('imp', 'phi', ('imp', ~B2, 'phi'))),
 ('wff', ('imp', ('imp', 'phi', ~B2), ('imp', 'phi', 'phi'))),
 ('proof', ~P1, ('imp', 'phi', ('imp', ~B2, 'phi'))),
 ('wff', 'phi'),
 ('wff', ~B2),
 ('wff', 'phi')]

In [142]:
idProofGame.rotateRight()
idProofGame.rotateRight()
idProofGame.rotateRight()
idProofGame.state

[('wff', 'phi'),
 ('wff', ~B2),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', ~B2)),
 ('wff', ('imp', 'phi', 'phi')),
 ('proof', ~P0, ('imp', 'phi', ~B2)),
 ('wff', ('imp', 'phi', ('imp', ~B2, 'phi'))),
 ('wff', ('imp', ('imp', 'phi', ~B2), ('imp', 'phi', 'phi'))),
 ('proof', ~P1, ('imp', 'phi', ('imp', ~B2, 'phi')))]

In [143]:
idProofGame.applyRule('simplification')
idProofGame.state

[('wff', 'phi'),
 ('wff', ~B3),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', ~B3)),
 ('wff', ('imp', 'phi', 'phi')),
 ('proof', ~P0, ('imp', 'phi', ~B3)),
 ('wff', ('imp', 'phi', ('imp', ~B3, 'phi'))),
 ('wff', ('imp', ('imp', 'phi', ~B3), ('imp', 'phi', 'phi'))),
 ('wff', 'phi'),
 ('wff', ~B3)]

In [144]:
idProofGame.rotateRight()
idProofGame.rotateRight()
idProofGame.rotateRight()
idProofGame.rotateRight()
idProofGame.state

[('wff', ('imp', 'phi', ('imp', ~B3, 'phi'))),
 ('wff', ('imp', ('imp', 'phi', ~B3), ('imp', 'phi', 'phi'))),
 ('wff', 'phi'),
 ('wff', ~B3),
 ('wff', 'phi'),
 ('wff', ~B3),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', ~B3)),
 ('wff', ('imp', 'phi', 'phi')),
 ('proof', ~P0, ('imp', 'phi', ~B3))]

In [145]:
idProofGame.applyRule('simplification')
idProofGame.state

[('wff', ('imp', 'phi', ('imp', ('imp', ~B4, 'phi'), 'phi'))),
 ('wff', ('imp', ('imp', 'phi', ('imp', ~B4, 'phi')), ('imp', 'phi', 'phi'))),
 ('wff', 'phi'),
 ('wff', ('imp', ~B4, 'phi')),
 ('wff', 'phi'),
 ('wff', ('imp', ~B4, 'phi')),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', ('imp', ~B4, 'phi'))),
 ('wff', ('imp', 'phi', 'phi')),
 ('wff', 'phi'),
 ('wff', ~B4)]

In [146]:
idProofGame.applyRule('phiWff')
idProofGame.state

[('wff', ('imp', 'phi', ('imp', ('imp', 'phi', 'phi'), 'phi'))),
 ('wff',
  ('imp', ('imp', 'phi', ('imp', 'phi', 'phi')), ('imp', 'phi', 'phi'))),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', 'phi')),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', 'phi')),
 ('wff', 'phi'),
 ('wff', ('imp', 'phi', ('imp', 'phi', 'phi'))),
 ('wff', ('imp', 'phi', 'phi')),
 ('wff', 'phi')]

In [147]:
while idProofGame.state != []:
    idProofGame.applyRule('phiWff')
    idProofGame.applyRule('impWff')

idProofGame.state

[]

In [152]:
idProofGame.startingState

('proof', ~P, ('imp', 'phi', 'phi'))

In [153]:
idProofGame.finalState()

('proof',
 ('mp',
  ('imp', 'phi', ('imp', 'phi', 'phi')),
  ('imp', 'phi', 'phi'),
  ('simp', 'phi', 'phi'),
  ('mp',
   ('imp', 'phi', ('imp', ('imp', 'phi', 'phi'), 'phi')),
   ('imp', ('imp', 'phi', ('imp', 'phi', 'phi')), ('imp', 'phi', 'phi')),
   ('simp', 'phi', ('imp', 'phi', 'phi')),
   ('frege', 'phi', ('imp', 'phi', 'phi'), 'phi'))),
 ('imp', 'phi', 'phi'))

In [177]:
S = lam('x', lam('y', lam('z', app(app('y','x'), app('z','x')))))
K = lam('x', lam('y', 'x'))
I = lam('x', 'x')
SKIGame = ProofGame(eq(app(S, app(K, K)), I), LambdaTheory)

In [178]:
SKIGame.state

[('eq',
  ('app',
   ('lam',
    'x',
    ('lam', 'y', ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
   ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
  ('lam', 'x', 'x'))]

In [179]:
SKIGame.applyRule('etaEquality')
SKIGame.state

[('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~x0),
  ('app', ('lam', 'x', 'x'), ~x0))]

In [180]:
SKIGame.applyRule('transitivity')
SKIGame.state

[('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~x0),
  ~B1),
 ('eq', ~B1, ('app', ('lam', 'x', 'x'), ~x0))]

In [181]:
SKIGame.applyRule('symmetry')
SKIGame.state

[('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~x0),
  ~A2),
 ('eq', ('app', ('lam', 'x', 'x'), ~x0), ~A2)]

In [182]:
SKIGame.applyRule('beta1')
SKIGame.state

[('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~B3),
  ~B3)]

In [190]:
SKIGame.applyRule('transitivity')
SKIGame.state

[('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~C11),
  ~B11),
 ('eq', ~B11, ~C11)]

In [191]:
SKIGame.rotateRight()
SKIGame.state

[('eq', ~B11, ~C11),
 ('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~C11),
  ~B11)]

In [196]:
SKIGame.applyRule('beta1')
SKIGame.state

[('eq', ~B11, ~C11),
 ('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~C11),
  ~B11)]

In [174]:
SKIGame.rotateRight()
SKIGame.rotateRight()
SKIGame.state

[('ev', 'x'),
 ('exp', ~B3),
 ('ev', ~B3),
 ('eq',
  ('app',
   ('app',
    ('lam',
     'x',
     ('lam',
      'y',
      ('lam', 'z', ('app', ('app', 'y', 'x'), ('app', 'z', 'x'))))),
    ('app', ('lam', 'x', ('lam', 'y', 'x')), ('lam', 'x', ('lam', 'y', 'x')))),
   ~B3),
  ~B3)]

In [175]:
SKIGame.DVSC

[(~A0, ~x0), (~B0, ~x0)]

In [ ]:
    'varExp': self.varExp,
                      'xEv': self.xEv,
                      'yEv': self.yEv,
                      'zEv': self.zEv,
                      'uEv': self.uEv,
                      'vEv': self.vEv,
                      'wEv': self.wEv,
                      'XEv': self.XEv,
                      'YEv': self.YEv,
                      'ZEv': self.ZEv,
                      'UEv': self.UEv,
                      'VEv': self.VEv,
                      'WEv': self.WEv,
                      'lamExp': self.lamExp,
                      'appExp': self.appExp,
                      'transitivity': self.transitivity,
                      'equalityOfApplication': self.equalityOfApplication,
                      'equalityOfLambdas': self.equalityOfLambdas,
                      'etaEquality': self.etaEquality,
                      'beta1': self.beta1,
                      'beta2': self.beta2,
                      'beta3': self.beta3,
                      'beta4': self.beta4,
                      'beta5': self.beta5